Transfer Learning Using VGG16

Importing libraries

In [1]:
import tensorflow as tf

In [2]:

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [3]:
ROOT_PATH='E:/Projects/Dog_breed_classification'

Change the directory to root path

In [4]:
import os
os.chdir(ROOT_PATH)
os.getcwd()

'E:\\Projects\\Dog_breed_classification'

Since the data was downloaded from kaggle the format isnt as expected for the project so we mannually fixed the data and then imported here